Imprting libraries

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar


# To Import parquets file, pyarrow or fastparquet module is neccessary

In [2]:
!pip install pyarrow

In [3]:
#reading downloaded parquets file
df = pd.read_parquet('~/data/fhv_tripdata_2021-01.parquet')
#df

# Questions 1
<br>Read the data for January. How many records are there?

Ans:1154112

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154112 entries, 0 to 1154111
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1154112 non-null  object        
 1   pickup_datetime         1154112 non-null  datetime64[ns]
 2   dropOff_datetime        1154112 non-null  datetime64[ns]
 3   PUlocationID            195845 non-null   float64       
 4   DOlocationID            991892 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1153227 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 61.6+ MB


In [5]:

#Extracting some date features

df['month'] = df['dropOff_datetime'].dt.month.astype('int') 

df['hour'] = df['dropOff_datetime'].dt.hour.astype('int')

df['dayOfWeek'] = df['dropOff_datetime'].dt.dayofweek.astype('int')

#weekend or not 
df['weekend'] = df['dayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)


bins = [0,4,8,12,16,20,24]
labels = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
df['day_session'] = pd.cut(df['hour'], bins=bins, labels=labels, include_lowest=True)

#pubic hiday


#seasn

def do_season_of_date(date):
    year = str(date.year)
    seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year,infer_datetime_format=True),
              'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year,infer_datetime_format=True),
              'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year,infer_datetime_format=True)}
    if date in seasons['spring']:
        return 'spring'
    if date in seasons['summer']:
        return 'summer'
    if date in seasons['autumn']:
        return 'autumn'
    else:
        return 'winter'


In [6]:
#This Season Column was removed since the data range is only Jan and feb (winter)
#df['season']=df['dropOff_datetime'].apply(do_season_of_date)

In [7]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,month,hour,dayOfWeek,weekend,day_session
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,1,0,4,0,Late Night
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,1,1,4,0,Late Night
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,1,1,4,0,Late Night
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,1,0,4,0,Late Night
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,1,0,4,0,Late Night
...,...,...,...,...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,1,23,6,1,Night
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,2,0,0,0,Late Night
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,1,23,6,1,Night
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,2,0,0,0,Late Night


In [8]:
#Adding Public holiday
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df['dropOff_datetime'].min(),
                        end=df['dropOff_datetime'].max())
df['is_holiday'] = np.where(df['dropOff_datetime'].isin(holidays),1,0)

df


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,month,hour,dayOfWeek,weekend,day_session,is_holiday
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,1,0,4,0,Late Night,0
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,1,1,4,0,Late Night,0
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,1,1,4,0,Late Night,0
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,1,0,4,0,Late Night,0
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,1,0,4,0,Late Night,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,1,23,6,1,Night,0
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,2,0,0,0,Late Night,0
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,1,23,6,1,Night,0
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,2,0,0,0,Late Night,0


In [9]:
df['is_holiday'].value_counts()


0    1154111
1          1
Name: is_holiday, dtype: int64

In [10]:
df[['pickup_datetime', 'dropOff_datetime']]

,pickup_datetime,dropOff_datetime
0,2021-01-01 00:27:00,2021-01-01 00:44:00
1,2021-01-01 00:50:00,2021-01-01 01:07:00
2,2021-01-01 00:01:00,2021-01-01 01:51:00
3,2021-01-01 00:13:09,2021-01-01 00:21:26
4,2021-01-01 00:38:31,2021-01-01 00:53:44
...,...,...
1154107,2021-01-31 23:43:03,2021-01-31 23:51:48
1154108,2021-01-31 23:50:27,2021-02-01 00:48:03
1154109,2021-01-31 23:13:46,2021-01-31 23:29:58
1154110,2021-01-31 23:58:03,2021-02-01 00:17:29


# Question 2

Whats the average trip duration in January?
To compute the average trip in jan we first determine the duration for each trip which is the difference 
between each dropoff time and pickuptime(NB:It should contain the duration of a ride in minutes).

Ans:19.167

In [11]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [12]:
df.duration.mean()

19.1672240937939

In [13]:
df.describe()

,PUlocationID,DOlocationID,month,hour,dayOfWeek,weekend,is_holiday,duration
count,195845.000000,991892.000000,1.154112e+06,1.154112e+06,1.154112e+06,1.154112e+06,1.154112e+06,1.154112e+06
mean,139.859690,135.898030,1.000127e+00,1.249498e+01,2.938534e+00,2.492878e-01,8.664670e-07,1.916722e+01
std,74.991382,80.474902,1.435978e-02,5.323140e+00,1.923324e+00,4.326009e-01,9.308421e-04,3.986922e+02
min,1.000000,1.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.666667e-02
25%,75.000000,67.000000,1.000000e+00,9.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,7.766667e+00
50%,143.000000,132.000000,1.000000e+00,1.200000e+01,3.000000e+00,0.000000e+00,0.000000e+00,1.340000e+01
75%,206.000000,213.000000,1.000000e+00,1.600000e+01,4.000000e+00,0.000000e+00,0.000000e+00,2.228333e+01
max,265.000000,265.000000,1.100000e+01,2.300000e+01,6.000000e+00,1.000000e+00,1.000000e+00,4.233710e+05


In [14]:
df.count()

dispatching_base_num      1154112
pickup_datetime           1154112
dropOff_datetime          1154112
PUlocationID               195845
DOlocationID               991892
SR_Flag                         0
Affiliated_base_number    1153227
month                     1154112
hour                      1154112
dayOfWeek                 1154112
weekend                   1154112
day_session               1154112
is_holiday                1154112
duration                  1154112
dtype: int64

we remove outliers & keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [15]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [16]:
df.describe()

,PUlocationID,DOlocationID,month,hour,dayOfWeek,weekend,is_holiday,duration
count,182818.000000,961919.000000,1.109826e+06,1.109826e+06,1.109826e+06,1.109826e+06,1.109826e+06,1.109826e+06
mean,139.861633,135.689147,1.000108e+00,1.246275e+01,2.946074e+00,2.519710e-01,9.010421e-07,1.624725e+01
std,74.763788,80.349678,1.039776e-02,5.330160e+00,1.927287e+00,4.341449e-01,9.492324e-04,1.155150e+01
min,1.000000,1.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,75.000000,67.000000,1.000000e+00,9.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,7.850000e+00
50%,146.000000,132.000000,1.000000e+00,1.200000e+01,3.000000e+00,0.000000e+00,0.000000e+00,1.323333e+01
75%,206.000000,213.000000,1.000000e+00,1.600000e+01,5.000000e+00,1.000000e+00,0.000000e+00,2.146667e+01
max,265.000000,265.000000,2.000000e+00,2.300000e+01,6.000000e+00,1.000000e+00,1.000000e+00,6.000000e+01


In [17]:
df.count()

dispatching_base_num      1109826
pickup_datetime           1109826
dropOff_datetime          1109826
PUlocationID               182818
DOlocationID               961919
SR_Flag                         0
Affiliated_base_number    1109053
month                     1109826
hour                      1109826
dayOfWeek                 1109826
weekend                   1109826
day_session               1109826
is_holiday                1109826
duration                  1109826
dtype: int64

Question 3
How many records did you drop?

Ans:44286

In [18]:
1154112 - 1109826

44286

In [19]:
df.isnull().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               927008
DOlocationID               147907
SR_Flag                   1109826
Affiliated_base_number        773
month                           0
hour                            0
dayOfWeek                       0
weekend                         0
day_session                     0
is_holiday                      0
duration                        0
dtype: int64

# Question 3
What's the factions of missing values for the pickup location ID? (Or the fraction of "-1"s after you filled the NAs)

In [20]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)

/tmp/ipykernel_5156/1051174452.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PUlocationID'] = df['PUlocationID'].fillna(-1)
/tmp/ipykernel_5156/1051174452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOlocationID'] = df['DOlocationID'].fillna(-1)


In [21]:
df.isnull().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID                    0
DOlocationID                    0
SR_Flag                   1109826
Affiliated_base_number        773
month                           0
hour                            0
dayOfWeek                       0
weekend                         0
day_session                     0
is_holiday                      0
duration                        0
dtype: int64

In [22]:
total = 1109826
pu = 927008

print(pu/total*100)

83.52732770722618


In [23]:
#Encoding categrical varriabe day_session
df['day_session']=df['day_session'].astype('category')
df["day_session"] = df["day_session"].cat.codes


/tmp/ipykernel_5156/516213263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_session']=df['day_session'].astype('category')
/tmp/ipykernel_5156/516213263.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["day_session"] = df["day_session"].cat.codes


In [24]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,month,hour,dayOfWeek,weekend,day_session,is_holiday,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,1,0,4,0,0,0,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,1,1,4,0,0,0,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,1,0,4,0,0,0,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,1,0,4,0,0,0,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,1,1,4,0,0,0,9.050000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,1,23,6,1,5,0,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,2,0,0,0,0,0,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,1,23,6,1,5,0,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,2,0,0,0,0,0,19.433333


In [25]:
from sklearn.feature_extraction import DictVectorizer

In [26]:
categorical = ['PUlocationID', 'DOlocationID']
numerical = ['duration']

In [27]:
df[categorical] = df[categorical].astype(str)

/tmp/ipykernel_5156/1767203464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [28]:
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [29]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [31]:
target = 'duration'
y_train = df[target].values

In [32]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107206316

In [33]:
df_val = pd.read_parquet('~/data/fhv_tripdata_2021-02.parquet')
df_val['duration'] = df_val.dropOff_datetime - df_val.pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

#Extracting sme date features

df_val['month'] = df_val['dropOff_datetime'].dt.month.astype('int') 

df_val['hour'] = df_val['dropOff_datetime'].dt.hour.astype('int')

df_val['dayOfWeek'] = df_val['dropOff_datetime'].dt.dayofweek.astype('int')

#weekend or Not
df_val['weekend'] =df_val['dayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)


bins = [0,4,8,12,16,20,24]
labels = ['Late Night', 'Early Morning','Morning','Noon','Eve','Night']
df_val['day_session'] = pd.cut(df_val['hour'], bins=bins, labels=labels, include_lowest=True)

#Encoding categrical varriabe day_session
df_val['day_session']=df_val['day_session'].astype('category')
df_val["day_session"] = df_val["day_session"].cat.codes

#Adding Public holiday
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df_val['dropOff_datetime'].min(),
                        end=df_val['dropOff_datetime'].max())
df_val['is_holiday'] = np.where(df_val['dropOff_datetime'].isin(holidays),1,0)



df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
df_val[categorical] = df_val[categorical].astype(str)

In [34]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [35]:
y_val = df_val[target].values

In [36]:
y_pred = lr.predict(X_val)

In [37]:
mean_squared_error(y_val, y_pred, squared=False)

11.36443467538724

References
https://practicaldatascience.co.uk/data-science/how-to-engineer-date-features-using-pandas
https://stackoverflow.com/questions/29688899/pandas-checking-if-a-date-is-a-holiday-and-assigning-boolean-value